<a href="https://colab.research.google.com/github/dimakpa/DL_Projects/blob/main/tif_test_v_2_preprocessing_tif_to_folders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
from torchvision.models import efficientnet_b6 as efficientnet_b6
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
import imgaug as ia
from imgaug import augmenters as iaa
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
# %matplotlib inline
import random
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from statistics import mean
from torch.optim import lr_scheduler
from PIL import Image
import csv
from skimage.measure import regionprops
from skimage.transform import resize

Функция, которая превращает .tif в файл со снимками, на которых размечены узлы


In [ ]:
# Папки с тифами разных тирадсов
paths_with_tifs_2_3 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/tirads2-3/'
paths_with_tifs_4 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/tirads4/'
paths_with_tifs_5 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/tirads5/'

# Куда сохраняем папки разных тирадсов
paths_to_save_tifs_2_3 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/data_after_preprocessing/tirads2-3/'
paths_to_save_tifs_4 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/data_after_preprocessing/tirads4/'
paths_to_save_tifs_5 = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/data_for_test/data_after_preprocessing/tirads5/'


# path_tif = 'C:/Users/dimak/Desktop/Work/Laba/dataset/CLASSIFICATION_new_dataset_study_17.10.23/study_019_TIRADS4_long.tif'

Соберем названия тифов в папках

In [ ]:
def get_pathes(path):
    x_pathes = []
    y_pathes = []
    for patient in os.listdir(path):
        # print(patient)
        for tif_name in os.listdir(path + patient + '/'):
            if 'mask' in tif_name:

                try:
                  if(tif_name[0] != 's'):
                    N = int(tif_name.split('_')[1][6])
                  else:
                    N = int(tif_name.split('_')[2][6])
                except IndexError:
                  N = 5 # Ищем данный файл в df и и сопоставляем TIRADS в y_paths
#                   N = search_tirads(tif_name, df_train, df_valid)

                y_pathes.append([patient + '/' + tif_name, N, tif_name]) #Тирадс считаю чуть по-другому
                x_pathes.append(patient + '/' + tif_name.replace('_mask', ''))
    return x_pathes, y_pathes

In [ ]:
x_pathes, y_pathes = get_pathes(paths_with_tifs_2_3)
y_pathes[0]

['study_001/study_001_TIRADS3_cross_mask.tif',
 3,
 'study_001_TIRADS3_cross_mask.tif']

Подгружаем тиф и разбиваем на фотки.

Создаем лист листов фоток


In [ ]:
def get_images(x_pathes, y_pathes, path):

    images = []
    masks = []
    for x_path, y_path in zip(x_pathes, y_pathes):
        if y_path[1] != 0:
          # Загрузчик файла

          new_images, new_masks = load_tif(x_path, y_path, path)

          images.append(new_images)
          masks.append(new_masks)

    return images, masks

In [ ]:
def load_tif(x_path, y_path, path, step=4):
    images = []
    masks = []
    image = Image.open(path + x_path)
    mask = Image.open(path + y_path[0])
    i = 0
    j = 0
    while True:
        try:
            mask.seek(i)
            mask_array = np.array(mask)
            image.seek(i)
            image_array = np.array(image)
            if len(np.unique(np.concatenate(mask_array, axis=0))) > 1: #Проверка, что маска не полностью черная
                if j % step == 0:
                    masks.append([mask_array, y_path[1], y_path[2]])
                    images.append(image_array)
                j += 1
            else:
                j = 0
            i += 1
        except EOFError:
            break
    return images, masks

In [ ]:
images, masks = get_images(x_pathes, y_pathes, paths_with_tifs_2_3)

In [ ]:
images[0]

[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 

In [ ]:
masks[0][0][0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

Теперь для каждого тифа создадим папку и туда сохраним все получившиеся фото

In [ ]:
def preprocessing2(roi: object):
    if np.sum(roi) == 0:
        minr, minc, maxr, maxc = [0, 0, 256, 256]
    else:
        region = regionprops(roi)[0]
        minr, minc, maxr, maxc = region.bbox
    dim1_center, dim2_center = [(maxr + minr) // 2, (maxc + minc) // 2]
    max_length = max(maxr - minr, maxc - minc)
    max_lengthl = int((256 / 256) * 80)
    preprocess1 = int((256 / 256) * 19)
    pp22 = int((256 / 256) * 31)
    if max_length > max_lengthl:
        ex_pixel = preprocess1 + max_length // 2
    else:
        ex_pixel = pp22 + max_length // 2
    dim1_cut_min = dim1_center - ex_pixel
    dim1_cut_max = dim1_center + ex_pixel
    dim2_cut_min = dim2_center - ex_pixel
    dim2_cut_max = dim2_center + ex_pixel
    if dim1_cut_min < 0:
        dim1_cut_min = 0
    if dim2_cut_min < 0:
        dim2_cut_min = 0
    if dim1_cut_max > 256:
        dim1_cut_max = 256
    if dim2_cut_max > 256:
        dim2_cut_max = 256
    return [dim1_cut_min, dim1_cut_max, dim2_cut_min, dim2_cut_max]

In [ ]:
def save_pngs(folder_path, images, masks, bias):
    tirads_list = []
    names_list = []
    j = 0
    for i in range(len(images)):
        path = os.path.join(folder_path, masks[i][0][2][:-9])
        os.mkdir(path)
#         print('Folder created...')
        for k in range(len(images[i])):
            cut_image = resize(images[i][k], (256, 256), order=3)
            cut_image = (cut_image * 255).astype(np.uint8)
            cut_mask = np.array(Image.fromarray(masks[i][k][0]).resize((256, 256)))
            dim1_cut_min, dim1_cut_max, dim2_cut_min, dim2_cut_max = preprocessing2(cut_mask)
            image_roi = cut_image[dim1_cut_min:dim1_cut_max, dim2_cut_min:dim2_cut_max]
            mask_roi = cut_mask[dim1_cut_min:dim1_cut_max, dim2_cut_min:dim2_cut_max]

            image_save_path = f'{path}/{j + bias}.PNG'
            Image.fromarray(image_roi).convert(mode='L').save(image_save_path) # 1-канальное изображение
            j+=1


Для каждого тифа создали папку с его изображениями

In [ ]:
x_pathes2_3, y_pathes2_3 = get_pathes(paths_with_tifs_2_3)
images2_3, masks2_3 = get_images(x_pathes2_3, y_pathes2_3, paths_with_tifs_2_3)
save_pngs(paths_to_save_tifs_2_3, images2_3, masks2_3, bias=0)

x_pathes4, y_pathes4 = get_pathes(paths_with_tifs_4)
images4, masks4 = get_images(x_pathes4, y_pathes4, paths_with_tifs_4)
save_pngs(paths_to_save_tifs_4, images4, masks4, bias=0)

x_pathes5, y_pathes5 = get_pathes(paths_with_tifs_5)
images5, masks5 = get_images(x_pathes5, y_pathes5, paths_with_tifs_5)
save_pngs(paths_to_save_tifs_5, images5, masks5, bias=0)

Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder created...
Folder cre

Теперь для каждой папки(тифа) посчитаем предикт